# Generate Forecasts and Test PKLS

1. Download IBM Forecast data
2. Save IBM Forecast data to database
3. Import PKLs and generate BL Forecasts
4. Compute statistics against IBM
5. Save statistics in database

In [1]:
from Namelist import db_info_local as db_info
from Namelist import Account_Station

import sys
sys.path.insert(0, '/Users/mborrus/Benchmark/GCP/ds-common/')
import API_Function_Library.Forecast_Functions as api_bl
import API_Function_Library.SQL_Functions as SQL_bl
from olm import *

import sqlalchemy as db
from sqlalchemy import create_engine
import pickle

from Namelist import Training_vars


In [2]:
AS_Index = 0;
AccountUid = Account_Station.AccountUid[AS_Index]
StationUid = Account_Station.StationUid[AS_Index]
Provider = Account_Station.Provider[AS_Index]

hostname = db_info['hostname']
dbname = db_info['dbname']
uname = db_info['uname']
pwd = db_info['pwd']

# Create SQLAlchemy engine to connect to MySQL Database
engine = db.create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(host=hostname, db=dbname, user=uname, pw=pwd))

# in for data from database/table, out for data to database/table 
connection_in = engine.connect() 
connection_out = SQL_bl.create_db_connection(hostname, uname, pwd, dbname) # Connect to the Database

###############

MySQL Database connection successful


#### 1. Download IBM Forecast data

In [3]:
IBM_Forecasts = api_bl.get_IBM_Forecasts(AccountUid,StationUid)
IBM_Forecasts

,Lead_Time,Local_datetime,PRECIPITATION_QUANTITY,TEMPERATURE,WIND_DIRECTION,WIND_SPEED,RELATIVE_HUMIDITY,stationUid,identifier
0,0,2022-12-19 13:00:00-08:00,0.0,18,302,2,24,c8a9d548-7fc4-40bb-954e-389988526593,80318
1,1,2022-12-19 14:00:00-08:00,0.0,18,303,3,26,c8a9d548-7fc4-40bb-954e-389988526593,80318
2,2,2022-12-19 15:00:00-08:00,0.0,17,303,3,32,c8a9d548-7fc4-40bb-954e-389988526593,80318
3,3,2022-12-19 16:00:00-08:00,0.0,15,302,3,43,c8a9d548-7fc4-40bb-954e-389988526593,80318
4,4,2022-12-19 17:00:00-08:00,0.0,12,320,2,57,c8a9d548-7fc4-40bb-954e-389988526593,80318
...,...,...,...,...,...,...,...,...,...
355,355,2023-01-03 08:00:00-08:00,0.0,10,98,2,62,c8a9d548-7fc4-40bb-954e-389988526593,80318
356,356,2023-01-03 09:00:00-08:00,0.0,12,122,1,51,c8a9d548-7fc4-40bb-954e-389988526593,80318
357,357,2023-01-03 10:00:00-08:00,0.0,14,121,2,44,c8a9d548-7fc4-40bb-954e-389988526593,80318
358,358,2023-01-03 11:00:00-08:00,0.0,16,270,2,40,c8a9d548-7fc4-40bb-954e-389988526593,80318


#### 2. Save IBM Forecast data to database



In [4]:
IBM_Forecasts.to_sql('IBM_Forecast_Data_Temp', engine, index=False, if_exists = 'replace')

##### Merge data with full table

name = "IBM_Forecast";
query_merge = '''
INSERT INTO %s_Data
SELECT %s_Data_Temp.*
FROM %s_Data_Temp
WHERE NOT EXISTS(SELECT * FROM %s_Data 
WHERE %s_Data_Temp.Lead_Time = %s_Data.Lead_Time 
AND %s_Data_Temp.Local_datetime = %s_Data.Local_datetime 
AND %s_Data_Temp.stationUid = %s_Data.stationUid);
''' % (name,name,name,name,name,name,name,name,name,name)

SQL_bl.execute_query(connection_out, query_merge) # Execute our defined query

Query successful


#### 3. Import PKLs and generate BL Forecasts

In [4]:
metadata = db.MetaData()
Pkl_table = db.Table('Pkl_Performance_Data', metadata, autoload=True, autoload_with=engine)

In [11]:
Training_var = Training_vars[1]
query = db.select([Pkl_table]).where(Pkl_table.columns.stationUid == StationUid)
Station_Data = SQL_bl.query_to_df(connection_in, query)

In [86]:
index = 306

station = Station_Data.loc[[index]]

pkl_test_filename = station.pkl_file_name.item()

with open('./pkls/'+pkl_test_filename, 'rb') as f:
    trainedmodel = pickle.load(f)
pkl_test_filename

'2a6997a7-a9a2-4b2e-b87a-656d6e85cbaf_c8a9d548-7fc4-40bb-954e-389988526593_RELATIVE_HUMIDITY_2022-12-19_1.pkl'

In [87]:
import numpy as np
import pandas as pd
def Forecast_From_Model(input_pkl, Input_Forecast_Data, Training_var):
    Input_Forecast_Data=Input_Forecast_Data.dropna()
    XForecast=Input_Forecast_Data['%s' % (Training_var)].to_numpy()
    XForecast=XForecast.reshape(XForecast.shape[0],1)

    # Create benchmark FORECASTS
    Yhat=Elm_predict(input_pkl, XForecast)
    Yhat=np.transpose(Yhat)
    df_Yhat = pd.DataFrame(Yhat,columns = ['forecast'],index=Input_Forecast_Data.Local_datetime)
    df_Yhat = df_Yhat.reset_index()
    df_Yhat['Local_datetime'] = pd.to_datetime(df_Yhat['Local_datetime'])

    Q1 = input_pkl['q1']
    Q3 = input_pkl['q3']
    IQR = Q3 - Q1

    print((Q1 - 1.5 * IQR).item())
    print((Q3 + 1.5 * IQR).item())
   
    lower_bound = (Q1 - 1.5 * IQR).item()
    upper_bound = (Q3 + 1.5 * IQR).item()
    
    #save data before outlier detection
    forecast_df = pd.DataFrame();
    forecast_df[['Local_datetime', 'dirty_forecast']] = df_Yhat; 
    forecast_df['IBM_forecast'] = Input_Forecast_Data['%s' % (Training_var)];

    #This is slightly different than what happens in the real code
    Outlier_Conditions = (df_Yhat['forecast'] > upper_bound) | (df_Yhat['forecast'] < lower_bound)
    df_Yhat.loc[Outlier_Conditions,'forecast'] = Input_Forecast_Data.loc[Outlier_Conditions,'%s' % (Training_var)]
    print("outliers: " + str(Outlier_Conditions.sum()))
    forecast_df['forecast'] = df_Yhat['forecast'];
    forecast_df = forecast_df.reset_index()
    forecast_df = forecast_df.rename(columns={'index': 'Lead_Time'})
    forecast_df['stationUid'] = Input_Forecast_Data['stationUid'];
    forecast_df['metric'] = Training_var
    outliers = Outlier_Conditions.sum()
    return(forecast_df,outliers)

Training_var = station.Metric.item()
forecasts, outliers = Forecast_From_Model(trainedmodel,IBM_Forecasts, Training_var)
forecasts['pkl_file_name'] =  pkl_test_filename;

outliers_df = pd.DataFrame();
outliers_df[['stationUid','metric','pkl_file_name']]=forecasts[['stationUid','metric','pkl_file_name']].loc[[1]]
outliers_df['Outlier_Count']=outliers

week_og = station.weeks.item()
day_og = station.Model_Date.item()
days_since = abs(day_og - datetime.date.today()).days

outliers_df[['weeks','Model_Date','Days_Since']]=[[week_og,day_og,days_since]]
outliers_df



-23.67500000000002
170.68500000000003
outliers: 22


,stationUid,metric,pkl_file_name,Outlier_Count,weeks,Model_Date,Days_Since
1,c8a9d548-7fc4-40bb-954e-389988526593,RELATIVE_HUMIDITY,2a6997a7-a9a2-4b2e-b87a-656d6e85cbaf_c8a9d548-...,22,1,2022-12-19,0


In [63]:
station.Metric

10    TEMPERATURE
Name: Metric, dtype: object

In [74]:
forecasts.to_sql('BL_Forecast_Data_Temp', engine, index=False, if_exists = 'replace')

##### Merge data with full table

name = "BL_Forecast";
query_merge = '''
INSERT INTO %s_Data
SELECT %s_Data_Temp.*
FROM %s_Data_Temp
WHERE NOT EXISTS(SELECT * FROM %s_Data 
WHERE %s_Data_Temp.Lead_Time = %s_Data.Lead_Time 
AND %s_Data_Temp.Local_datetime = %s_Data.Local_datetime 
AND %s_Data_Temp.pkl_file_name = %s_Data.pkl_file_name
AND %s_Data_Temp.metric = %s_Data.metric);
''' % (name,name,name,name,name,name,name,name,name,name,name,name)

SQL_bl.execute_query(connection_out, query_merge) # Execute our defined query

Query successful


In [73]:
outliers_df.to_sql('Outlier_Data_Temp', engine, index=False, if_exists = 'replace')

##### Merge data with full table

name = "Outlier";
query_merge = '''
INSERT INTO %s_Data
SELECT %s_Data_Temp.*
FROM %s_Data_Temp
WHERE NOT EXISTS(SELECT * FROM %s_Data 
WHERE %s_Data_Temp.pkl_file_name = %s_Data.pkl_file_name
AND %s_Data_Temp.metric = %s_Data.metric);
''' % (name,name,name,name,name,name,name,name)

SQL_bl.execute_query(connection_out, query_merge) # Execute our defined query

Query successful


In [9]:
unique_pkls

array(['2a6997a7-a9a2-4b2e-b87a-656d6e85cbaf_c8a9d548-7fc4-40bb-954e-389988526593_TEMPERATURE_2022-12-06_1.pkl',
       '2a6997a7-a9a2-4b2e-b87a-656d6e85cbaf_c8a9d548-7fc4-40bb-954e-389988526593_TEMPERATURE_2022-12-06_2.pkl',
       '2a6997a7-a9a2-4b2e-b87a-656d6e85cbaf_c8a9d548-7fc4-40bb-954e-389988526593_TEMPERATURE_2022-12-06_3.pkl',
       '2a6997a7-a9a2-4b2e-b87a-656d6e85cbaf_c8a9d548-7fc4-40bb-954e-389988526593_TEMPERATURE_2022-12-06_4.pkl',
       '2a6997a7-a9a2-4b2e-b87a-656d6e85cbaf_c8a9d548-7fc4-40bb-954e-389988526593_TEMPERATURE_2022-12-06_5.pkl',
       '2a6997a7-a9a2-4b2e-b87a-656d6e85cbaf_c8a9d548-7fc4-40bb-954e-389988526593_TEMPERATURE_2022-12-06_6.pkl',
       '2a6997a7-a9a2-4b2e-b87a-656d6e85cbaf_c8a9d548-7fc4-40bb-954e-389988526593_TEMPERATURE_2022-12-06_7.pkl',
       '2a6997a7-a9a2-4b2e-b87a-656d6e85cbaf_c8a9d548-7fc4-40bb-954e-389988526593_TEMPERATURE_2022-12-06_8.pkl',
       '2a6997a7-a9a2-4b2e-b87a-656d6e85cbaf_c8a9d548-7fc4-40bb-954e-389988526593_TEMPERATURE_20